In [1]:
# Import Dependencies
import requests
from config import api_key
import pandas as pd  
from sqlalchemy import create_engine
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
import numpy as np
from sqlalchemy import func
from sqlalchemy import distinct
import json
import psycopg2
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
cloud_engine = create_engine(f"postgresql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
cloud_conn = cloud_engine.connect()

ca_school_data = pd.read_sql('select * from ml_master_clean', cloud_conn)

ca_school_data.head()

,Zipcode,Total Students Enrolled,Median Household Income,Median Income BLACK/AA,Median Income INDIGENOUS PEOPLE,Median Income ASIAN,Median Income HAW/PAC ISLANDER,Median Income OTHER RACE,Median Income MIXED RACE,Median Income WHITE/NOT LATINO,...,Substance Abuse Count,Violent Count,Miscellaneous Count,School Name,School Rating,School Address,City,Grades,Students Per Teacher,District
0,91901,244.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,The Heights Charter,7.0,"2710 Alpine Boulevard, Suite E",Alpine,K-8,15:1,Dehesa Elementary
1,91901,333.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Boulder Oaks Elementary,6.0,2320 Tavern Road,Alpine,1-5,24:1,Alpine Union Elementary
2,91901,438.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Joan MacQueen Middle,6.0,2001 Tavern Road,Alpine,6-8,20:1,Alpine Union Elementary
3,91901,387.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Shadow Hills Elementary,6.0,8770 Harbison Canyon Road,Alpine,K-5,20:1,Alpine Union Elementary
4,91901,158.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,317.0,50.0,52.0,Alpine Elementary,4.0,1850 Alpine Boulevard,Alpine,1-5,20:1,Alpine Union Elementary


Four New Columns for Crime ratios are created which will standardize the count with the Zipcode population
The new columns created are 'Theft Count Ratio', 'Substance Abuse Count Ratio', 'Violent Count Ratio' and 'Miscellaneous Count Ratio'

In [3]:
ca_school_data['Theft Count Ratio'] = ca_school_data['Theft Count'] / ca_school_data['Population Total']
ca_school_data['Substance Abuse Count Ratio'] = ca_school_data['Substance Abuse Count'] / ca_school_data['Population Total']
ca_school_data['Violent Count Ratio'] = ca_school_data['Violent Count'] / ca_school_data['Population Total']
ca_school_data['Miscellaneous Count Ratio'] = ca_school_data['Miscellaneous Count'] / ca_school_data['Population Total']
ca_school_data.head(75)

,Zipcode,Total Students Enrolled,Median Household Income,Median Income BLACK/AA,Median Income INDIGENOUS PEOPLE,Median Income ASIAN,Median Income HAW/PAC ISLANDER,Median Income OTHER RACE,Median Income MIXED RACE,Median Income WHITE/NOT LATINO,...,School Rating,School Address,City,Grades,Students Per Teacher,District,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,244.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,7.0,"2710 Alpine Boulevard, Suite E",Alpine,K-8,15:1,Dehesa Elementary,0.004269,0.017129,0.002702,0.002810
1,91901,333.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,2320 Tavern Road,Alpine,1-5,24:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
2,91901,438.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,2001 Tavern Road,Alpine,6-8,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
3,91901,387.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,8770 Harbison Canyon Road,Alpine,K-5,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
4,91901,158.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,4.0,1850 Alpine Boulevard,Alpine,1-5,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,91915,332.0,101760.0,109750.0,0.0,101328.0,0.0,82159.0,121719.0,117525.0,...,4.0,1949 Discovery Falls Drive,Chula Vista,6-8,14:1,SBC - High Tech High,0.003226,0.000319,0.001489,0.002340
71,91916,211.0,69821.0,0.0,0.0,0.0,0.0,0.0,0.0,69955.0,...,6.0,24842 Viejas Boulevard,Descanso,K-6,21:1,Mountain Empire Unified,0.002483,0.000000,0.000000,0.000000
72,91932,896.0,46659.0,40521.0,74773.0,41701.0,0.0,37527.0,36649.0,57344.0,...,6.0,650 Imperial Beach Boulevard,Imperial Beach,K-8,20:1,South Bay Union,0.004632,0.006402,0.004406,0.003126
73,91932,1585.0,46659.0,40521.0,74773.0,41701.0,0.0,37527.0,36649.0,57344.0,...,4.0,505 Elm Avenue,Imperial Beach,9-12,25:1,Sweetwater Union High,0.004632,0.006402,0.004406,0.003126


Removing the Crime Actual Counts from the dataframe i.e. 
1. Theft Count
2. Substance Abuse Count
3. Violent Count
4. Miscellaneous Count

In [4]:
ca_school_data = ca_school_data.drop(labels=["Theft Count", "Substance Abuse Count","Violent Count","Miscellaneous Count"], axis=1)
ca_school_data.head(75)

,Zipcode,Total Students Enrolled,Median Household Income,Median Income BLACK/AA,Median Income INDIGENOUS PEOPLE,Median Income ASIAN,Median Income HAW/PAC ISLANDER,Median Income OTHER RACE,Median Income MIXED RACE,Median Income WHITE/NOT LATINO,...,School Rating,School Address,City,Grades,Students Per Teacher,District,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,244.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,7.0,"2710 Alpine Boulevard, Suite E",Alpine,K-8,15:1,Dehesa Elementary,0.004269,0.017129,0.002702,0.002810
1,91901,333.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,2320 Tavern Road,Alpine,1-5,24:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
2,91901,438.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,2001 Tavern Road,Alpine,6-8,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
3,91901,387.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,6.0,8770 Harbison Canyon Road,Alpine,K-5,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
4,91901,158.0,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,88039.0,...,4.0,1850 Alpine Boulevard,Alpine,1-5,20:1,Alpine Union Elementary,0.004269,0.017129,0.002702,0.002810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,91915,332.0,101760.0,109750.0,0.0,101328.0,0.0,82159.0,121719.0,117525.0,...,4.0,1949 Discovery Falls Drive,Chula Vista,6-8,14:1,SBC - High Tech High,0.003226,0.000319,0.001489,0.002340
71,91916,211.0,69821.0,0.0,0.0,0.0,0.0,0.0,0.0,69955.0,...,6.0,24842 Viejas Boulevard,Descanso,K-6,21:1,Mountain Empire Unified,0.002483,0.000000,0.000000,0.000000
72,91932,896.0,46659.0,40521.0,74773.0,41701.0,0.0,37527.0,36649.0,57344.0,...,6.0,650 Imperial Beach Boulevard,Imperial Beach,K-8,20:1,South Bay Union,0.004632,0.006402,0.004406,0.003126
73,91932,1585.0,46659.0,40521.0,74773.0,41701.0,0.0,37527.0,36649.0,57344.0,...,4.0,505 Elm Avenue,Imperial Beach,9-12,25:1,Sweetwater Union High,0.004632,0.006402,0.004406,0.003126


Removing the following columns from the dataframe i.e. 
1. latitude
2. longitude
3. School Address
4. City
5. Grades


In [5]:
ca_school_drop = ca_school_data.drop(labels=["latitude", "longitude","School Name","School Address","City","Grades","Median Income BLACK/AA","Median Income INDIGENOUS PEOPLE",
                "Median Income ASIAN", "Median Income HAW/PAC ISLANDER","Median Income OTHER RACE","Median Income MIXED RACE","Median Income WHITE/NOT LATINO",
                "Median Income HISPANIC/LATINO","District","Students Per Teacher","Theft Count Ratio","Substance Abuse Count Ratio","Violent Count Ratio",
                "Miscellaneous Count Ratio"], axis=1)
ca_school_drop.head(75)

,Zipcode,Total Students Enrolled,Median Household Income,o25_10th_grade_attained_ratio,o25_11th_grade_attained_ratio,o25_12th_grade_started_ratio,o25_high_school_completed_ratio,o25_ged_attained_ratio,o25_less_one_yr_college_ratio,o25_more_one_yr_college_ratio,...,Population Total,Population white ratio,Population black/aa ratio,Population indigenous ratio,Population asian ratio,Population haw/pac islander ratio,Population other ratio,Population mixed ratio,Population hispanic/latino ratio,School Rating
0,91901,244.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,18507.0,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,7.0
1,91901,333.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,18507.0,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0
2,91901,438.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,18507.0,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0
3,91901,387.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,18507.0,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,6.0
4,91901,158.0,79859.0,0.012959,0.024338,0.027341,0.193915,0.030897,0.057764,0.242513,...,18507.0,0.769114,0.013184,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,91915,332.0,101760.0,0.008781,0.002117,0.023789,0.131959,0.006165,0.052248,0.176174,...,28209.0,0.215569,0.058173,0.000674,0.208302,0.000000,0.010777,0.033961,0.472544,4.0
71,91916,211.0,69821.0,0.006108,0.020942,0.006981,0.166667,0.015707,0.099476,0.263525,...,1611.0,0.838610,0.006828,0.000000,0.014277,0.004345,0.000000,0.015518,0.120422,6.0
72,91932,896.0,46659.0,0.022927,0.017494,0.036062,0.187116,0.032360,0.087408,0.208430,...,26555.0,0.334589,0.030465,0.002975,0.082282,0.008887,0.000226,0.039578,0.500998,6.0
73,91932,1585.0,46659.0,0.022927,0.017494,0.036062,0.187116,0.032360,0.087408,0.208430,...,26555.0,0.334589,0.030465,0.002975,0.082282,0.008887,0.000226,0.039578,0.500998,4.0


In [6]:
ca_school_drop.dtypes


Zipcode                                      int64
Total Students Enrolled                    float64
Median Household Income                    float64
o25_10th_grade_attained_ratio              float64
o25_11th_grade_attained_ratio              float64
o25_12th_grade_started_ratio               float64
o25_high_school_completed_ratio            float64
o25_ged_attained_ratio                     float64
o25_less_one_yr_college_ratio              float64
o25_more_one_yr_college_ratio              float64
o25_assoc_degree_completed_ratio           float64
o25_bach_degree_completed_ratio            float64
o25_masters_degree_completed_ratio         float64
o25_professional_degree_completed_ratio    float64
o25_doctoral_degree_completed_ratio        float64
married ratio                              float64
female hoh ratio                           float64
male hoh ratio                             float64
non family ratio                           float64
Population Total               

In [7]:
ca_school_cat = list(ca_school_drop.dtypes[ca_school_drop.dtypes == "object"].index)
ca_school_cat

[]

In [8]:
# Split our preprocessed data into our features and target arrays
X = ca_school_drop.drop("School Rating", axis=1).values
y = ca_school_drop["School Rating"].values.reshape(-1, 1)
#target_names = ["negative", "positive"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:

# Fit a model, and then print a classification report
clf = RandomForestClassifier(random_state=99).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         2.0       0.17      0.11      0.13         9
         3.0       0.23      0.24      0.23        21
         4.0       0.12      0.12      0.12        25
         5.0       0.19      0.33      0.25        21
         6.0       0.38      0.33      0.35        27
         7.0       0.31      0.24      0.27        21
         8.0       0.38      0.33      0.35        27
         9.0       0.39      0.41      0.40        17
        10.0       0.00      0.00      0.00         4

    accuracy                           0.26       174
   macro avg       0.22      0.21      0.21       174
weighted avg       0.27      0.26      0.26       174

Training Score: 0.9980842911877394
Testing Score: 0.26436781609195403


In [11]:
clf = ExtraTreesClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         2.0       0.14      0.11      0.12         9
         3.0       0.14      0.14      0.14        21
         4.0       0.12      0.12      0.12        25
         5.0       0.19      0.33      0.25        21
         6.0       0.38      0.33      0.35        27
         7.0       0.27      0.19      0.22        21
         8.0       0.36      0.33      0.35        27
         9.0       0.39      0.41      0.40        17
        10.0       0.00      0.00      0.00         4

    accuracy                           0.25       174
   macro avg       0.20      0.20      0.20       174
weighted avg       0.25      0.25      0.24       174

Training Score: 0.9980842911877394
Testing Score: 0.2471264367816092


In [12]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(random_state=1).fit(X_train_scaled, y_train)
print(classification_report(y_test, y_pred))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         2.0       0.14      0.11      0.12         9
         3.0       0.14      0.14      0.14        21
         4.0       0.12      0.12      0.12        25
         5.0       0.19      0.33      0.25        21
         6.0       0.38      0.33      0.35        27
         7.0       0.27      0.19      0.22        21
         8.0       0.36      0.33      0.35        27
         9.0       0.39      0.41      0.40        17
        10.0       0.00      0.00      0.00         4

    accuracy                           0.25       174
   macro avg       0.20      0.20      0.20       174
weighted avg       0.25      0.25      0.24       174

Training Score: 0.2547892720306513
Testing Score: 0.25862068965517243


C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [13]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from matplotlib import pyplot as plt

In [14]:
def test_model(model, data):
    X_train_scaled, X_test_scaled, y_train, y_test = data
    reg = model.fit(X_train_scaled, y_train)
    print(f'Model: {type(reg).__name__}')
    print(f'Train score: {reg.score(X_train_scaled, y_train)}')
    print(f'Test Score: {reg.score(X_test_scaled, y_test)}\n')
    plt.show()
    df = pd.DataFrame()
    df['true'] = y_test.flatten()
    df['pred'] = reg.predict(X_test_scaled)
    df['MSE'] = (df['pred']-df['true'])**2
    df['SD'] = np.sqrt((df['MSE']))
    df.describe()
    return df.describe()

In [17]:
data = [X_train_scaled, X_test_scaled, y_train, y_test]

test_model(LinearRegression(), data)

test_model(KNeighborsRegressor(), data)
test_model(RandomForestRegressor(n_estimators=10, max_depth=4), data)
test_model(ExtraTreesRegressor(n_estimators=10, max_depth=4), data)
test_model(AdaBoostRegressor(), data)
test_model(SVR(C=1.0, epsilon=0.2), data)

Model: LinearRegression
Train score: 0.4354187808884691
Test Score: 0.3929155957046784

Model: KNeighborsRegressor
Train score: 0.5530593883382637
Test Score: 0.370988146610586

Model: RandomForestRegressor
Train score: 0.5669821358775369
Test Score: 0.43020797080260964

Model: ExtraTreesRegressor
Train score: 0.48838180610471515
Test Score: 0.404568338815977



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model: AdaBoostRegressor
Train score: 0.49298042023607647
Test Score: 0.4114297754105606

Model: SVR
Train score: 0.49411854801144306
Test Score: 0.4073307745715389



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.835710,2.866956,1.353406
std,2.205748,1.359549,4.055882,1.020408
min,1.000000,3.646012,0.000067,0.008193
25%,4.000000,4.846057,0.301830,0.549352
50%,6.000000,5.401635,1.452449,1.205174
75%,8.000000,7.025714,3.640612,1.907825
max,10.000000,9.180065,26.342018,5.132448


In [ ]:
test_model(LinearRegression(), data)

Model: LinearRegression
Train score: 0.4354187808884691
Test Score: 0.3929155957046784



,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.751041,2.936688,1.387535
std,2.205748,1.386525,4.220371,1.008603
min,1.000000,2.034690,0.001203,0.034690
25%,4.000000,4.644536,0.389746,0.624297
50%,6.000000,5.519931,1.481409,1.217127
75%,8.000000,6.775683,3.844965,1.960855
max,10.000000,9.694717,28.592174,5.347165


In [ ]:
test_model(KNeighborsRegressor(), data)

Model: KNeighborsRegressor
Train score: 0.5530593883382637
Test Score: 0.370988146610586



,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.912644,3.042759,1.379310
std,2.205748,1.500890,4.305730,1.070912
min,1.000000,2.800000,0.000000,0.000000
25%,4.000000,4.650000,0.360000,0.600000
50%,6.000000,5.600000,1.440000,1.200000
75%,8.000000,7.200000,4.000000,2.000000
max,10.000000,9.000000,29.160000,5.400000


In [18]:
test_model(RandomForestRegressor(n_estimators=10, max_depth=4), data)

Model: RandomForestRegressor
Train score: 0.5654954788934196
Test Score: 0.4328557377620893



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.848309,2.743483,1.343945
std,2.205748,1.375804,3.796378,0.970934
min,1.000000,3.342748,0.000470,0.021684
25%,4.000000,4.881878,0.315281,0.561481
50%,6.000000,5.381098,1.161850,1.077891
75%,8.000000,6.971192,3.605904,1.898921
max,10.000000,8.981533,25.514450,5.051183


In [19]:
test_model(ExtraTreesRegressor(n_estimators=10, max_depth=4), data)

Model: ExtraTreesRegressor
Train score: 0.481887060027122
Test Score: 0.41271539800312995



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.841965,2.840909,1.337000
std,2.205748,1.322021,4.378568,1.029285
min,1.000000,3.873544,0.000371,0.019255
25%,4.000000,4.855434,0.406916,0.637897
50%,6.000000,5.088901,1.283893,1.133071
75%,8.000000,7.033665,3.243199,1.800886
max,10.000000,9.078938,27.426988,5.237078


In [ ]:
test_model(AdaBoostRegressor(), data)


Model: AdaBoostRegressor
Train score: 0.4874446229956506
Test Score: 0.37330138620113384



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.920650,3.031569,1.424674
std,2.205748,1.120680,3.943745,1.003824
min,1.000000,3.965517,0.000000,0.000000
25%,4.000000,4.888889,0.424472,0.651515
50%,6.000000,5.783761,1.534208,1.238524
75%,8.000000,6.920000,4.559496,2.135179
max,10.000000,7.936170,27.820454,5.274510


In [ ]:
test_model(SVR(C=1.0, epsilon=0.2), data)

Model: SVR
Train score: 0.49411854801144306
Test Score: 0.4073307745715389



C:\Users\GPSchool\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,true,pred,MSE,SD
count,174.000000,174.000000,174.000000,174.000000
mean,5.781609,5.835710,2.866956,1.353406
std,2.205748,1.359549,4.055882,1.020408
min,1.000000,3.646012,0.000067,0.008193
25%,4.000000,4.846057,0.301830,0.549352
50%,6.000000,5.401635,1.452449,1.205174
75%,8.000000,7.025714,3.640612,1.907825
max,10.000000,9.180065,26.342018,5.132448
